#Importing libraries

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris, load_wine, load_breast_cancer, make_classification
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, validation_curve
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import graphviz
from collections import Counter
import math
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

#DataFrame

In [22]:
data = {
    "Day": [1,2,3,4,5,6,7,8,9,10,11,12,13,14],
    "Outlook": ["Sunny","Sunny","Overcast","Rain","Rain","Rain","Overcast","Sunny","Sunny","Rain","Sunny","Overcast","Overcast","Rain"],
    "Temperature": ["Hot","Hot","Hot","Mild","Cool","Cool","Cool","Mild","Cool","Mild","Mild","Mild","Hot","Mild"],
    "Humidity": ["High","High","High","High","Normal","Normal","Normal","High","Normal","Normal","Normal","High","Normal","High"],
    "Wind": ["Weak","Strong","Weak","Weak","Weak","Strong","Strong","Weak","Weak","Weak","Strong","Strong","Weak","Strong"],
    "Play Tennis": ["No","No","Yes","Yes","Yes","No","Yes","No","Yes","Yes","Yes","Yes","Yes","No"]
}

df = pd.DataFrame(data)
print(df)

    Day   Outlook Temperature Humidity    Wind Play Tennis
0     1     Sunny         Hot     High    Weak          No
1     2     Sunny         Hot     High  Strong          No
2     3  Overcast         Hot     High    Weak         Yes
3     4      Rain        Mild     High    Weak         Yes
4     5      Rain        Cool   Normal    Weak         Yes
5     6      Rain        Cool   Normal  Strong          No
6     7  Overcast        Cool   Normal  Strong         Yes
7     8     Sunny        Mild     High    Weak          No
8     9     Sunny        Cool   Normal    Weak         Yes
9    10      Rain        Mild   Normal    Weak         Yes
10   11     Sunny        Mild   Normal  Strong         Yes
11   12  Overcast        Mild     High  Strong         Yes
12   13  Overcast         Hot   Normal    Weak         Yes
13   14      Rain        Mild     High  Strong          No


#Calculating Entropy

In [23]:
days = 14
pYes = 9/days
pNo = 4/days
entropy = -pYes * math.log2(pYes) - pNo * math.log2(pNo)
print(entropy)

0.9261634981262887


#Info gain for each

In [24]:
p_yes_sunny, p_no_sunny = 2/5, 3/5
p_yes_rain, p_no_rain = 3/5, 2/5

ent_sunny = -p_yes_sunny * math.log2(p_yes_sunny) - p_no_sunny * math.log2(p_no_sunny)
ent_overcast = 0
ent_rain = -(p_yes_rain * math.log2(p_yes_rain)) - (p_no_rain * math.log2(p_no_rain))

weighted_outlook = 5/14 * ent_sunny + 4/14 * ent_overcast + 5/14 * ent_rain
inf_outlook = entropy - weighted_outlook


p_yes_hot, p_no_hot = 2/4, 2/4
p_yes_mild, p_no_mild = 4/6, 2/6
p_yes_cool, p_no_cool = 3/4, 1/4

ent_mild = -(p_yes_mild * math.log2(p_yes_mild)) - (p_no_mild * math.log2(p_no_mild))
ent_cool = -(p_yes_cool * math.log2(p_yes_cool) + p_no_cool * math.log2(p_no_cool))
ent_hot = -(p_yes_hot * math.log2(p_yes_hot)) - (p_no_hot * math.log2(p_no_hot))

weighted_temp = (4/14) * ent_hot + (6/14) * ent_mild + (4/14) * ent_cool
inf_temp = entropy - weighted_temp


p_yes_high, p_no_high = 3/7, 4/7
p_yes_normal, p_no_normal = 6/7, 1/7

ent_high = -(p_yes_high*math.log2(p_yes_high) + p_no_high*math.log2(p_no_high))
ent_nornal = -(p_yes_normal*math.log2(p_yes_normal) + p_no_normal*math.log2(p_no_normal))

weighted_humidity = 7/14 * ent_high + 7/14 * ent_nornal
inf_humidity = entropy - weighted_humidity


ent_strong = 1
p_weak_yes, p_weak_no = 6/8, 2/8
ent_weak = -(p_weak_yes*math.log2(p_weak_yes) + p_weak_no*math.log2(p_weak_no))

weighted_wind = 6/14 * ent_strong + 8/14 * ent_weak
inf_wind = entropy - weighted_wind

print(f"Outlook: {inf_outlook}, temp: {inf_temp}, humidity: {inf_humidity}, wind: {inf_wind}")

Outlook: 0.23262735923009692, temp: 0.015100105114612461, humidity: 0.13771304081799918, wind: 0.03400456986392708


#Gini impurity

In [25]:
def gini(p_yes, p_no):
    return 1 - (p_yes**2 + p_no**2)

p_yes_sunny, p_no_sunny = 2/5, 3/5
p_yes_overcast, p_no_overcast = 4/4, 0/4
p_yes_rain, p_no_rain = 3/5, 2/5
gini_sunny = gini(p_yes_sunny, p_no_sunny)
gini_overcast = gini(p_yes_overcast, p_no_overcast)
gini_rain = gini(p_yes_rain, p_no_rain)
weighted_gini_outlook = (5/14)*gini_sunny + (4/14)*gini_overcast + (5/14)*gini_rain
print(f"Gini Outlook: {weighted_gini_outlook:.4f}")

p_yes_hot, p_no_hot = 2/4, 2/4
p_yes_mild, p_no_mild = 4/6, 2/6
p_yes_cool, p_no_cool = 3/4, 1/4
gini_hot = gini(p_yes_hot, p_no_hot)
gini_mild = gini(p_yes_mild, p_no_mild)
gini_cool = gini(p_yes_cool, p_no_cool)
weighted_gini_temp = (4/14)*gini_hot + (6/14)*gini_mild + (4/14)*gini_cool
print(f"Gini Temperature: {weighted_gini_temp:.4f}")

p_yes_high, p_no_high = 3/7, 4/7
p_yes_normal, p_no_normal = 6/7, 1/7
gini_high = gini(p_yes_high, p_no_high)
gini_normal = gini(p_yes_normal, p_no_normal)
weighted_gini_humidity = (7/14)*gini_high + (7/14)*gini_normal
print(f"Gini Humidity: {weighted_gini_humidity:.4f}")

p_yes_strong, p_no_strong = 1/2, 1/2
p_yes_weak, p_no_weak = 6/8, 2/8
gini_strong = gini(p_yes_strong, p_no_strong)
gini_weak = gini(p_yes_weak, p_no_weak)
weighted_gini_wind = (2/14)*gini_strong + (8/14)*gini_weak
print(f"Gini Wind: {weighted_gini_wind:.4f}")

Gini Outlook: 0.3429
Gini Temperature: 0.4405
Gini Humidity: 0.3673
Gini Wind: 0.2857


#Constructing Trees

In [26]:
class TreeNode:
    """
    Represents a node in the decision tree.

    Can be either an internal node with a splitting rule or a leaf node
    with a prediction.
    """

    def __init__(self, is_leaf=False):

        self.is_leaf = is_leaf

        self.feature_index = None
        self.threshold = None
        self.feature_value = None

        self.prediction = None
        self.class_counts = None

        self.left = None
        self.right = None
        self.children = {}

        self.samples = 0
        self.impurity = 0.0
        self.depth = 0

    def __repr__(self):
        if self.is_leaf:
            return f"Leaf(prediction={self.prediction}, samples={self.samples})"
        else:
            return f"Node(feature={self.feature_index}, threshold={self.threshold}, samples={self.samples})"

root = TreeNode()
leaf = TreeNode(is_leaf=True)
print("Node classes created successfully!")
print(f"Root node: {root}")
print(f"Leaf node: {leaf}")

Node classes created successfully!
Root node: Node(feature=None, threshold=None, samples=0)
Leaf node: Leaf(prediction=None, samples=0)


#Impurity Measures Implementation

In [27]:
def calculate_entropy(y):
    """
    Calculate the entropy of a target variable.

    Entropy is a measure of the impurity or randomness of a set of labels.
    A set with perfect purity (all labels are the same) has entropy 0.
    A set with maximum impurity (labels are equally distributed) has higher entropy.

    Parameters:
    y: array-like, target labels

    Returns:
    float: calculated entropy
    """
    values, counts = np.unique(y, return_counts = True)
    probs = counts / counts.sum()
    return -np.sum(probs * np.log2(probs))

def calculate_gini_impurity(y):
    """
    Calculate the Gini impurity of a target variable.

    Gini impurity is another measure of the impurity or randomness of a set of labels.
    It represents the probability of incorrectly classifying a randomly chosen element
    in the dataset if it were randomly labeled according to the distribution of labels
    in the dataset.

    Parameters:
    y: array-like, target labels

    Returns:
    float: calculated Gini impurity
    """
    values, counts = np.unique(y, return_counts=True)
    probs = counts / counts.sum()
    return 1 - np.sum(probs**2)

def calculate_mse(y, y_pred):
    """
    Calculate the Mean Squared Error (MSE).

    MSE is a common metric for evaluating the performance of regression models.
    In the context of decision trees, it's used as an impurity measure for regression trees,
    representing the variance of the target values in a node.

    Parameters:
    y: array-like, true target values
    y_pred: array-like, predicted target values (can be the mean of y for a node)

    Returns:
    float: calculated MSE
    """
    mean_y = np.mean(y)
    return np.mean((y - mean_y) ** 2)

test_labels_pure = np.array([1, 1, 1, 1])
test_labels_mixed = np.array([1, 0, 1, 0])
test_labels_skewed = np.array([1, 1, 1, 0])

print("Testing impurity measures:")
print(f"Pure set entropy: {calculate_entropy(test_labels_pure)}")
print(f"Mixed set entropy: {calculate_entropy(test_labels_mixed)}")
print(f"Skewed set entropy: {calculate_entropy(test_labels_skewed)}")

print("\nGini impurity: ")
print("Gini for pure", calculate_gini_impurity(test_labels_pure))
print("Gini for mixed", calculate_gini_impurity(test_labels_mixed))
print("Gini for skewed", calculate_gini_impurity(test_labels_skewed))

print("\nMSE:")
print("MSE for pure", calculate_mse(test_labels_pure, test_labels_pure))
print("MSE for mixed", calculate_mse(test_labels_mixed, test_labels_mixed))
print("MSE for skewed", calculate_mse(test_labels_skewed, test_labels_skewed))

Testing impurity measures:
Pure set entropy: -0.0
Mixed set entropy: 1.0
Skewed set entropy: 0.8112781244591328

Gini impurity: 
Gini for pure 0.0
Gini for mixed 0.5
Gini for skewed 0.375

MSE:
MSE for pure 0.0
MSE for mixed 0.25
MSE for skewed 0.1875


#Feature Splitting Logic

In [28]:
def find_best_split_categorical(X_column, y, impurity_func):
    """
    Find the best split for a categorical feature.

    For categorical features, we try splitting on each possible value,
    creating one branch for that value and another for all other values.

    Parameters:
    X_column: array-like, values of a single categorical feature
    y: array-like, target labels
    impurity_func: function, impurity measure to use

    Returns:
    best_value: value to split on
    best_impurity_reduction: improvement in impurity
    left_indices: indices of samples going to left child
    right_indices: indices of samples going to right child
    """
    uniques = np.unique(X_column)
    best_value = None
    best_reduction = -1
    best_left_indices = None
    best_right_indices = None

    parent_impurity = impurity_func(y)

    for value in uniques:
        left_indices = np.where(X_column == value)[0]
        right_indices = np.where(X_column != value)[0]

        if len(left_indices) == 0 or len(right_indices) == 0:
            continue

        left_labels = y[left_indices]
        right_labels = y[right_indices]

        left_impurity = impurity_func(left_labels)
        right_impurity = impurity_func(right_labels)

        weighted_impurity = (len(left_labels)/len(y)) * left_impurity + (len(right_labels)/len(y)) * right_impurity
        reduction = parent_impurity - weighted_impurity

        if reduction > best_reduction:
            best_value = value
            best_reduction = reduction
            best_left_indices = left_indices
            best_right_indices = right_indices

    return best_value, best_reduction, best_left_indices, best_right_indices


def find_best_split_continuous(X_column, y, impurity_func):
    """
    Find the best split threshold for a continuous feature.

    We consider all possible thresholds (typically midpoints between
    consecutive sorted values) and choose the one that maximizes
    information gain.

    Parameters:
    X_column: array-like, values of a single continuous feature
    y: array-like, target labels
    impurity_func: function, impurity measure

    Returns:
    best_threshold: threshold value for split
    best_impurity_reduction: improvement in impurity
    left_indices: indices of samples with value <= threshold
    right_indices: indices of samples with value > threshold
    """
    X_sorted = np.sort(X_column)
    sorted_indices = np.argsort(X_column)
    y_sorted = y[sorted_indices]

    best_threshold = None
    best_reduction = -1
    best_left_indices = None
    best_right_indices = None

    parent_impurity = impurity_func(y)

    for i in range(1, len(X_column)):
        threshold = (X_sorted[i-1] + X_sorted[i]) / 2
        left_indices = sorted_indices[np.where(X_column[sorted_indices] <= threshold)[0]]
        right_indices = sorted_indices[np.where(X_column[sorted_indices] > threshold)[0]]

        if len(left_indices) == 0 or len(right_indices) == 0:
            continue

        left_labels = y[left_indices]
        right_labels = y[right_indices]

        left_impurity = impurity_func(left_labels)
        right_impurity = impurity_func(right_labels)

        weighted_impurity = (len(left_labels)/len(y)) * left_impurity + (len(right_labels)/len(y)) * right_impurity
        reduction = parent_impurity - weighted_impurity

        if reduction > best_reduction:
            best_threshold = threshold
            best_reduction = reduction
            best_left_indices = left_indices
            best_right_indices = right_indices

    return best_threshold, best_reduction, best_left_indices, best_right_indices

def variance(y):
    return np.var(y)

def find_best_feature_split(X, y, feature_types, impurity_func):
    """
    Find the best feature and split point across all features.

    This function orchestrates the search across all possible features
    and returns the overall best split for building the tree.

    Parameters:
    X: array-like, shape (n_samples, n_features), feature matrix
    y: array-like, target values
    feature_types: list, 'categorical' or 'continuous' for each feature
    impurity_func: function, impurity measure

    Returns:
    best_feature: index of best feature to split on
    best_split_info: dictionary with split details
    """
    best_gain = -1
    best_feature = None
    best_split = None
    current_impurity = impurity_func(y)
    n_samples, n_features = X.shape
    for feature_index in range(n_features):
        if feature_types[feature_index] == 'continuous':
            best_threshold, gain, left_mask, right_mask = find_best_split_continuous(X[:, feature_index], y, impurity_func)
            if gain is not None and gain > best_gain:
                best_gain = gain
                best_feature = feature_index
                best_split = {'type':'continuous','threshold':best_threshold,'left':left_mask,'right':right_mask}
        else:
            best_value, gain, left_mask, right_mask = find_best_split_categorical(X[:, feature_index], y, impurity_func)
            if gain is not None and gain > best_gain:
                best_gain = gain
                best_feature = feature_index
                best_split = {'type':'categorical','value':best_value,'left':left_mask,'right':right_mask}
    return best_feature, best_split


sample_X = np.array([[1, 2.5], [2, 1.8], [1, 3.2], [3, 2.1], [2, 2.9]])
sample_y = np.array([0, 0, 1, 1, 1])
feature_types = ['categorical', 'continuous']

print("Testing split finding...")

print("Categorical split:")
print(find_best_split_categorical(sample_X[:, 0], sample_y, calculate_entropy))

print("Continuous split:")
print(find_best_split_continuous(sample_X[:, 1], sample_y, calculate_entropy))

Testing split finding...
Categorical split:
(np.float64(3.0), np.float64(0.17095059445466854), array([3]), array([0, 1, 2, 4]))
Continuous split:
(np.float64(2.7), np.float64(0.4199730940219749), array([1, 3, 0]), array([4, 2]))


#Tree Building Algorithm

In [29]:
class DecisionTreeClassifier:

    """
    Custom implementation of Decision Tree Classifier.

    This implementation provides full control over the tree building process
    and helps you understand every decision made during construction.
    """

    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1,
                 impurity_measure='entropy', feature_types=None):

        """
        Initialize the decision tree classifier.

        Parameters:
        max_depth: int or None, maximum depth of the tree
        min_samples_split: int, minimum samples required to split a node
        min_samples_leaf: int, minimum samples required at a leaf node
        impurity_measure: str, 'entropy' or 'gini'
        feature_types: list, type of each feature ('categorical' or 'continuous')
        """

        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.feature_types = feature_types
        self.root = None
        if impurity_measure == 'entropy':
            self.impurity_func = calculate_entropy
        elif impurity_measure == 'gini':
            self.impurity_func = calculate_gini_impurity
        else:
            raise ValueError(f"Unknown impurity measure: {impurity_measure}")

    def _build_tree(self, X, y, depth=0):

        """
        Recursively build the decision tree.

        This is the core recursive algorithm that creates the tree structure
        by repeatedly finding the best splits and creating child nodes.

        Parameters:
        X: feature matrix for current subset
        y: target values for current subset
        depth: current depth in the tree
        node_indices: indices of samples at this node

        Returns:
        TreeNode: the constructed node (internal or leaf)
        """

        if len(np.unique(y)) == 1:
            leaf = TreeNode(is_leaf=True)
            leaf.prediction = np.unique(y)[0]
            leaf.samples = len(y)
            leaf.depth = depth
            return leaf
        if self.max_depth is not None and depth >= self.max_depth:
            leaf = TreeNode(is_leaf=True)
            leaf.prediction = np.bincount(y).argmax()
            leaf.samples = len(y)
            leaf.depth = depth
            return leaf
        if len(y) < self.min_samples_split:
            leaf = TreeNode(is_leaf=True)
            leaf.prediction = np.bincount(y).argmax()
            leaf.samples = len(y)
            leaf.depth = depth
            return leaf

        best_feature, best_split = find_best_feature_split(X, y, self.feature_types, self.impurity_func)

        if best_feature is None or len(best_split['left']) < self.min_samples_leaf or len(best_split['right']) < self.min_samples_leaf:
            leaf = TreeNode(is_leaf=True)
            leaf.prediction = np.bincount(y).argmax()
            leaf.samples = len(y)
            leaf.depth = depth
            return leaf

        node = TreeNode(is_leaf=False)
        node.feature_index = best_feature
        node.samples = len(y)
        node.depth = depth

        left_X, left_y = X[best_split['left']], y[best_split['left']]
        right_X, right_y = X[best_split['right']], y[best_split['right']]

        if best_split['type'] == 'categorical':
            node.feature_value = best_split['value']
        else:
            node.threshold = best_split['threshold']

        node.left = self._build_tree(left_X, left_y, depth + 1)
        node.right = self._build_tree(right_X, right_y, depth + 1)

        return node

    def fit(self, X, y):

        """
        Build the decision tree from training data.

        Parameters:
        X: array-like, shape (n_samples, n_features), training features
        y: array-like, shape (n_samples,), training labels
        """

        if self.feature_types is None and X.shape[1] > 0:
             raise ValueError("feature_types must be provided if X has multiple columns.")


        self.root = self._build_tree(X, y)
        return self

    def _predict_sample(self, x, node):

        """
        Predict the class for a single sample by traversing the tree.

        Parameters:
        x: single sample (feature vector)
        node: current node in traversal

        Returns:
        prediction: predicted class label
        """

        if node.is_leaf:
            return node.prediction
        if self.feature_types[node.feature_index] == 'categorical':
            if x[node.feature_index] == node.feature_value:
                return self._predict_sample(x, node.left)
            else:
                return self._predict_sample(x, node.right)
        else:
            if x[node.feature_index] <= node.threshold:
                return self._predict_sample(x, node.left)
            else:
                return self._predict_sample(x, node.right)


    def predict(self, X):
        """
        Predict classes for multiple samples.

        Parameters:
        X: array-like, shape (n_samples, n_features), test features

        Returns:
        predictions: array of predicted class labels
        """
        predictions = []
        for i in range(len(X)):
            predictions.append(self._predict_sample(X[i], self.root))
        return np.array(predictions)

X = np.array([
    [1, 2.5],
    [1, 3.0],
    [2, 1.8],
    [2, 2.0]
])
y = np.array([0, 0, 1, 1])
feature_types = ['categorical', 'continuous']

tree = DecisionTreeClassifier(max_depth=2, feature_types=feature_types, impurity_measure='entropy', min_samples_leaf=1)
tree.fit(X, y)
predictions = tree.predict(X)
print(predictions)

[0 0 1 1]


#Tree Visualization and Interpretation

In [30]:
def print_tree(node, feature_names=None, class_names=None, indent="", max_depth=10):
    """
    Print a text representation of the decision tree.

    This function creates a human-readable representation that shows
    the decision rules learned by the tree.

    Parameters:
    node: TreeNode, root of tree/subtree to print
    feature_names: list, names of features for readable output
    class_names: list, names of classes for readable output
    indent: str, current indentation level
    max_depth: int, maximum depth to print (prevents overly long output)
    """
    if node is None or max_depth == 0:
        return None

    if node.is_leaf:
        prediction_text = class_names[node.prediction] if class_names and node.prediction is not None and isinstance(node.prediction, int) and node.prediction < len(class_names) and node.prediction >= 0 else str(node.prediction)
        print(f"{indent}Leaf: Prediction={prediction_text}, Samples={node.samples}")

    else:
        feature_name = feature_names[node.feature_index] if feature_names and node.feature_index is not None else f"feature[{node.feature_index}]"
        if node.feature_value is not None:
            print(f"{indent}Node: {feature_name} == {node.feature_value}? Samples={node.samples}")
            print_tree(node.left, feature_names, class_names, indent + "  ", max_depth - 1)
            print_tree(node.right, feature_names, class_names, indent + "  ", max_depth - 1)
        elif node.threshold is not None:
            print(f"{indent}Node: {feature_name} <= {node.threshold}? Samples={node.samples}")
            print_tree(node.left, feature_names, class_names, indent + "  ", max_depth - 1)
            print_tree(node.right, feature_names, class_names, indent + "  ", max_depth - 1)
        else:
             print(f"{indent}Node: Split condition not defined? Samples={node.samples}")


def visualize_tree_structure(node, feature_names=None, class_names=None, ax=None, x=0.5, y=1.0, dx=0.2, dy=0.15):
    """
    Create a visual representation of the tree structure using Matplotlib.

    Generate a matplotlib-based visualization showing the tree structure,
    decision boundaries, and class distributions at each node.

    Parameters:
    node: TreeNode, root of tree/subtree to visualize
    feature_names: list, names of features for readable output
    class_names: list, names of classes for readable output
    ax: matplotlib Axes object (optional, for drawing on existing axes)
    x, y: coordinates for the current node
    dx, dy: spacing parameters for child nodes
    """

    if ax is None:
        fig, ax = plt.subplots(figsize=(12, 8))
        ax.set_axis_off()


    if node.is_leaf:
        prediction_text = class_names[node.prediction] if class_names and node.prediction is not None and isinstance(node.prediction, int) and node.prediction < len(class_names) and node.prediction >= 0 else str(node.prediction)
        text = f"Predict: {prediction_text}\nSamples: {node.samples}"
        ax.text(x, y, text, ha='center', va='center',
                bbox=dict(boxstyle="round", facecolor="lightblue", ec="black"))
        return

    feature_name = feature_names[node.feature_index] if feature_names and node.feature_index is not None else f"feature[{node.feature_index}]"
    if node.feature_value is not None:
        text = f"{feature_name} == {node.feature_value}?\nSamples: {node.samples}"
    elif node.threshold is not None:
        text = f"{feature_name} <= {node.threshold:.2f}?\nSamples: {node.samples}"
    else:
        text = f"Split condition not defined?\nSamples: {node.samples}"

    ax.text(x, y, text, ha='center', va='center', bbox=dict(boxstyle="round", facecolor="lightgreen", ec="black"))

    if node.left:
        child_x = x - dx
        child_y = y - dy
        ax.plot([x, child_x], [y - 0.05, child_y + 0.05], 'k-')
        visualize_tree_structure(node.left, feature_names, class_names, ax, child_x, child_y, dx / 2, dy)

    if node.right:
        child_x = x + dx
        child_y = y - dy
        ax.plot([x, child_x], [y - 0.05, child_y + 0.05], 'k-')
        visualize_tree_structure(node.right, feature_names, class_names, ax, child_x, child_y, dx / 2, dy)


def extract_decision_rules(node, feature_names=None, path="", rules_list=None):
    """
    Extract all decision rules from the tree as text.

    Convert the tree into a set of if-then rules that can be easily
    understood and potentially used outside the tree structure.

    Parameters:
    node: TreeNode, current node
    feature_names: list, feature names
    path: str, current path conditions
    rules_list: list, accumulator for rules

    Returns:
    list: all decision rules as strings
    """

    if rules_list is None:
        rules_list = []

    if node is None:
        return rules_list

    if node.is_leaf:
        rules_list.append(f"{path.strip(' AND ')} -> prediction: {node.prediction}")
        return rules_list

    if node.feature_index is not None:
        feature = feature_names[node.feature_index] if feature_names else f"feature[{node.feature_index}]"
        if node.feature_value is not None:
            extract_decision_rules(node.left, feature_names, path + f"{feature} == {node.feature_value} AND ", rules_list)
            extract_decision_rules(node.right, feature_names, path + f"{feature} != {node.feature_value} AND ", rules_list)
        elif node.threshold is not None:
            extract_decision_rules(node.left, feature_names, path + f"{feature} <= {node.threshold:.2f} AND ", rules_list)
            extract_decision_rules(node.right, feature_names, path + f"{feature} > {node.threshold:.2f} AND ", rules_list)
        else:
             # Handle cases where split condition is undefined
             extract_decision_rules(node.left, feature_names, path, rules_list)
             extract_decision_rules(node.right, feature_names, path, rules_list)


    return rules_list

#Regression Tree Implementation

In [31]:
class DecisionTreeRegressor:
    """
    Custom implementation of Decision Tree Regressor.

    Regression trees predict continuous values rather than discrete classes.
    The main differences are in the impurity measure (MSE instead of entropy)
    and prediction method (mean instead of majority vote).
    """

    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        """
        Initialize the decision tree regressor.
        """
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None

    def _build_tree(self, X, y, depth=0):
        """
        Build regression tree using MSE as splitting criterion.

        Key differences from classification:
        - Use MSE to measure impurity
        - Leaf predictions are mean of target values
        - Consider variance reduction instead of information gain
        """
        if len(y) <= self.min_samples_split or (self.max_depth and depth >= self.max_depth):
            leaf = TreeNode(is_leaf=True)
            leaf.prediction = y.mean()
            return leaf

        n_samples, n_features = X.shape
        best_feature, best_threshold, best_mse = None, None, float('inf')

        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for t in thresholds:
                left_mask = X[:, feature] <= t
                right_mask = ~left_mask
                if left_mask.sum() < self.min_samples_leaf or right_mask.sum() < self.min_samples_leaf:
                    continue
                mse_split = (np.var(y[left_mask]) * left_mask.sum() +
                             np.var(y[right_mask]) * right_mask.sum()) / n_samples
                if mse_split < best_mse:
                    best_mse = mse_split
                    best_feature = feature
                    best_threshold = t

        if best_feature is None:
            leaf = TreeNode(is_leaf=True)
            leaf.prediction = y.mean()
            return leaf

        node = TreeNode(is_leaf=False)
        node.feature_index = best_feature
        node.threshold = best_threshold
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask
        node.left = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        node.right = self._build_tree(X[right_mask], y[right_mask], depth + 1)
        return node

    def fit(self, X, y):
        """Train the regression tree."""
        self.root = self._build_tree(np.array(X), np.array(y))

    def _predict_sample(self, x, node):
        if node.is_leaf:
            return node.prediction
        if x[node.feature_index] <= node.threshold:
            return self._predict_sample(x, node.left)
        else:
            return self._predict_sample(x, node.right)

    def predict(self, X):
        return np.array([self._predict_sample(x, self.root) for x in X])

print("Testing regression tree...")
X = np.array([[1,2],[1,3],[2,1],[2,2]])
y = np.array([10, 12, 20, 22])
tree = DecisionTreeRegressor(max_depth=2)
tree.fit(X, y)
preds = tree.predict(X)
print(preds)

Testing regression tree...
[11. 11. 21. 21.]


#Advanced Features and Optimizations

In [32]:
def implement_pruning(tree_root, X_val, y_val, pruning_method='reduced_error'):
    """
    Implement tree pruning to reduce overfitting.

    Pruning removes subtrees that don't improve performance on validation data.
    This is crucial for creating trees that generalize well to new data.

    Parameters:
    tree_root: TreeNode, root of tree to prune
    X_val: validation features
    y_val: validation labels
    pruning_method: str, pruning strategy to use

    Returns:
    TreeNode: pruned tree root
    """
    if tree_root is None or tree_root.is_leaf:
        return tree_root

    tree_root.left = implement_pruning(tree_root.left, X_val, y_val, pruning_method)
    tree_root.right = implement_pruning(tree_root.right, X_val, y_val, pruning_method)

    if tree_root.left and tree_root.left.is_leaf and tree_root.right and tree_root.right.is_leaf:
        y_pred = []
        for xi in X_val:
            if tree_root.feature_value is not None:
                y_pred.append(tree_root.left.prediction if xi[tree_root.feature_index] == tree_root.feature_value else tree_root.right.prediction)
            else:
                y_pred.append(tree_root.left.prediction if xi[tree_root.feature_index] <= tree_root.threshold else tree_root.right.prediction)
        acc_with_subtree = np.mean(np.array(y_pred) == y_val)

        leaf_prediction = np.round(np.mean([tree_root.left.prediction, tree_root.right.prediction]))
        acc_as_leaf = np.mean(np.full_like(y_val, leaf_prediction) == y_val)

        if acc_as_leaf >= acc_with_subtree:
            leaf = TreeNode(is_leaf=True)
            leaf.prediction = leaf_prediction
            return leaf
    return tree_root


def implement_feature_importance(tree_root, n_features):
    """
    Calculate feature importance scores based on tree structure.

    Feature importance measures how much each feature contributes
    to decreasing impurity across all splits in the tree.

    Parameters:
    tree_root: TreeNode, root of trained tree
    n_features: int, number of features

    Returns:
    array: importance scores for each feature
    """
    importance = np.zeros(n_features)
    def traverse(node):
        if node is None or node.is_leaf:
            return
        importance[node.feature_index] += 1
        traverse(node.left)
        traverse(node.right)
    traverse(tree_root)
    if np.sum(importance) > 0:
        importance = importance / np.sum(importance)
    return importance

def implement_missing_value_handling(X, method='surrogate'):
    """
    Handle missing values in the dataset.

    Decision trees can handle missing values through surrogate splits
    or by treating missing as a separate category.

    Parameters:
    X: feature matrix with potential missing values
    method: str, method for handling missing values

    Returns:
    X_processed: processed feature matrix
    """
    X_new = X.copy()
    for i in range(X_new.shape[1]):
        col = X_new[:, i]
        missing = np.isnan(col)
        if np.any(missing):
            if method == 'mean':
                col[missing] = np.nanmean(col)
            elif method == 'zero':
                col[missing] = 0
            else:
                col[missing] = -1
            X_new[:, i] = col
    return X_new

print("Testing advanced features...")
X_val = np.array([[1,2],[2,1],[1,3]])
y_val = np.array([0,1,0])
tree_root = tree.root

tree_root = implement_pruning(tree_root, X_val, y_val)
importances = implement_feature_importance(tree_root, X.shape[1])
X_processed = implement_missing_value_handling(X)

print("Feature importances:", importances)
print("Processed X:\n", X_processed)

Testing advanced features...
Feature importances: [0. 0.]
Processed X:
 [[1 2]
 [1 3]
 [2 1]
 [2 2]]
